In [1]:
import rebound
import sys
#change the next line to reflect where you have downloaded the source code
sys.path.insert(0, '/Users/kvolk/Documents/GitHub/SBDynT/src')
import sbdynt as sbd


# *Tools to directly set up a rebound integration from JPL orbit fits*
These routines use JPL's small body database to pull best-fit orbits and (if you want clones) an orbit-fit  covariance matrix for a small body given its designation and initializes a rebound simulation instance with that object and the sun and planets for the epoch of the orbit fit 

see detailed-examples-of-JPL-and-Horizons-query-functions.ipynb for details on the JPL queries

**Example of setting up an integration for a single object**

Note that the rebound simulation will have the following units: <br /> 
distance:  au <br /> 
time: years (1 year==365.25 days) <br /> 
mass: solar masses <br /> 

The designation of the small body can be given in most common formats, here we use packed provisional format, but other formats for this example object would include <br /> 
small_body= '2014XT40' (unpacked provisional des without a space) <br /> 
small_body= '2014 XT40' (unpacked provisional des with a space)  <br /> 
small_body = '535167' (this object is also numbered, so this is the number) <br /> 
small_body = 'r5167' (that number can be packed or not) <br /> 

The most basic use of the sbd.**initialize_simulation** function will return a simulation with all the major planets initialized at the epoch of the small body's best-fit orbit:

In [2]:
small_body = 'K14X40T'
flag, epoch, sim = sbd.initialize_simulation(des=small_body)

if(flag):
    print("Sucessfully added ", small_body, "to a simulation at epoch ", epoch)
    print("the best fit clone has a heliocentric orbit of:")
    
    helio_orbit = sim.particles[small_body].orbit(sim.particles[0])
    print("a=",helio_orbit.a)
    print("e=",helio_orbit.e)
    print("i=",helio_orbit.inc)
    print("node=",helio_orbit.Omega)
    print("aperi=",helio_orbit.omega)
    print("mean anomaly=",helio_orbit.M)

    print("\n\nThe status of the rebound simulation instance generated is:")
    sim.status(showAllFields=False)

Sucessfully added  K14X40T to a simulation at epoch  2457217.5
the best fit clone has a heliocentric orbit of:
a= 39.39346545211934
e= 0.19068466911250045
i= 0.12809190830418987
node= -2.376471686736393
aperi= 3.0447520417387786
mean anomaly= 0.1496946219973534


The status of the rebound simulation instance generated is:
---------------------------------
REBOUND version:     	4.4.1
REBOUND built on:    	May  8 2024 00:22:37
Number of particles: 	10
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.012000
---------------------------------
<rebound.particle.Particle object at 0x304e07050, m=1.0 x=0.00345997337268094 y=0.0002804304906489453 z=-0.0001531944658273006 vx=0.0008360120491290403 vy=0.0022947255451278956 vz=-2.261460135168542e-05>
<rebound.particle.Particle object at 0x304e057d0, m=1.660120825450693e-07 x=0.15955259895299326 y=0.2679908518849033 z=0.007400627257518443 vx=-10.926050573217358 vy=5.58755384754587 vz=1.4588322762800312

**Specifying planets, adding clones, saving logs and initial conditions**

You can also specify the planets you want to include in the simulation (for example, the terrestrial planets aren't really needed if you are integrating a TNO!). This is done by passing a list of **planets** in the function call. Note that as a shortcut, you can also use planets=['outer'] to use just the giant planets. During the initialization, the masses of planets not included in the call are accounted for and added to the Sun, and the timestep of the simulation is set to a value reasonable for the closest-in included planet. **\*Note\*** that if you are simulating very close-in comets or near earth asteroids (closer-in than Mercury's orbit), you may need to reset the default timestep to an smaller value to ensure you are accurately resolving their perihelion passages!

If clones to sample the orbit uncertainy are desired, then the JPL orbit fit covariance matrix will be used to generate
them (the best-fit orbit will always be the first particle and then clones are added in addition to the best-fit orbit). Cloning will happen if **clones** > 0 in the sbd.initialize_simulation function call. If the clones parameter is not included in the function call, it is assumed that clones=0.

You can also choose to save the queried covariance matrix that was used for the cloning to a pickle file (useful if you run the same object in the future and want to compare a new orbit-fit to an older one). That is achieved using the **save_sbdb** parameter, which can be set to True (saves to a default file name of \<designation\>-\<date\>-.pkl) or to a string with your desired file name. The default location for where files will be saved is the directory you are currently in, but can also be specified using **datadir** (as is done in the example below).

Setting the **saveic** parameter to True will save the initialized rebound simulation instance to a simulation archive binary file with the name \<designation\>-ic.bin. You can also specify a string with a different file name. If the file already exists, the initial conditions will be appended.


Finally, you can specify whether you want to generate a log, either to screen or to a file. If **logfile**=True, log messages will be appended to \<designation\>-log.txt (in either the current directory of the datadir); logfile can also be set to a string with a desired file name for the log or set to 'screen' to print the log messages to the screen instead of a file.


In [4]:
small_body = 'K14X40T'
n_clones = 5
flag, epoch, sim = sbd.initialize_simulation(des=small_body,
                                             planets=['Jupiter','Saturn','Uranus','Neptune'],
                                             clones=n_clones,
                                             save_sbdb=True,
                                             saveic=True,
                                             datadir = 'outputs-from-example-notebooks',
                                             logfile='screen')

#print out the simulation status
if(flag):
    print("Sucessfully added ", small_body, "to a simulation at epoch ", epoch)
    sim.status(showAllFields=False)


Sucessfully added  K14X40T to a simulation at epoch  2457217.5
---------------------------------
REBOUND version:     	4.4.1
REBOUND built on:    	May  8 2024 00:22:37
Number of particles: 	11
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.400000
---------------------------------
<rebound.particle.Particle object at 0x304e05e50, m=1.0000059769986274 x=0.0034610601623145097 y=0.00027631918524403633 z=-0.00015320621456877916 vx=0.0008681058230389303 vy=0.0023024789190979255 vz=-2.337071977160678e-05>
<rebound.particle.Particle object at 0x304e05ed0, m=0.0009547919099366768 x=-4.640079009063582 y=2.709933685630652 z=0.09249943531696177 vx=-1.4224164878812873 vy=-2.2502849100954925 vz=0.04117886591796361>
<rebound.particle.Particle object at 0x304e05e50, m=0.0002858856700231729 x=-4.527081593730463 y=-8.893180388894034 z=0.33480063198292653 vx=1.7044567379097002 vy=-0.9303393869988008 vz=-0.05165803566676291>
<rebound.particle.Particle obje

You can proceed from here with regular rebound commands, but we have also included some tools
to directly run the rebound simulations, writing outputs to a simulation archive binary file. 

The simulation defaults to using rebound's mercurius integrator with for the timestep as set above based on the planets in the simulation. **Note that the simulation
does not check to see if a small body gets closer to the sun than the innermost included planet!!! This
means that the integration timestep might become unreasonable if the small body evolves too far inward.**
(Or you could be missing important perturbers, such as the terrestrial planets, if you initialized the 
simulation with only the outer planets! Check for such conditions when analyzing the output)

In [ ]:
#max time in the integration (years)
tmax = 1e6
#how often you want to save the simulation archive file
tout = 1e3


savefile = "K14X40T-1Myr-archive.bin"

#run the simulation, deleting any existing simulation archive
flag, sim = sbd.run_simulation(sim, tmax=tmax, tout=tout, archivefile=savefile,deletefile=True)

#if you want to use a whfast or ias15, you can specify that 
#flag, sim = sbd.run_simulation(sim, tmax=tmax, tout=tout,filename=savefile,deletefile=True,integrator='ias15')
#flag, sim = sbd.run_simulation(sim, tmax=tmax, tout=tout,filename=savefile,deletefile=True,integrator='whfast')

if(flag):
    print("Simulation ran to ", sim.t, " years")

**Example of setting up an integration with a list of small bodies**

(no clones allowed in this instance as the cloning procedure relies on orbit fits that have independent epochs!)



In [ ]:
# Example of setting up an integration with a list of small bodies (no clones allowed)

#Set up an empty simulation, specify the small body and number of clones
sim= rebound.Simulation()
# as above, designations can be packed, unpacked, numbers, comets, etc
list_of_sbodies = ['K14X40T','2016 SW50', '15760','29P','179P/Jedicke']

#initialize the simulation without specifying planets (it will thus include all the major planets)
flag, epoch, sim = sbd.initialize_simulation_at_epoch(des=list_of_sbodies, epoch=2457019.)

#print out the simulation status
if(flag):
    sim.status(showAllFields=False)


# **Example plots from the integration of K14X40T above**

There are two ways to read the simualtion archive file with the build in functions: by hash or by small body name (which will read in all the clones too if there are any)

In [ ]:
#first by hash, where the best-fit always has the hash small_body and clones are small_body+"_j" for clones 1 through j
small_body = 'K14X40T'
nclones=5
flag, abf, ebf, incbf, nodebf, aperibf, mabf, t = sbd.read_sa_by_hash(obj_hash = small_body, archivefile=savefile)

#you can read the planet evolution this way too, their hashes are their names in all lower case
#(though the routine will change your input to lowercase if you enter, e.g. "Jupiter")
flag, a_j, e_j, inc_j, node_j, aperi_j, ma_j, t = sbd.read_sa_by_hash(obj_hash = "jupiter", archivefile=savefile)

In [ ]:
#now reading all the clones (the best fit will be indexed 0, then clones)
flag, a, e, inc, node, aperi, ma, t = sbd.read_sa_for_small_body(small_body=small_body, archivefile=savefile,nclones=nclones)

In [ ]:
#plot just the best-fit orbit (no clones)
#figfile can be omitted if you don't want to save the figure
flag, fig = sbd.plot_aei(small_body=small_body,a=a,e=e,inc=inc,t=t,nclones=0,figfile='example-K14X40T-bf-aei.png')

In [ ]:
#make a two-column plot that also shows the clones
flag, fig2 = sbd.plot_aei(small_body=small_body,a=a,e=e,inc=inc,t=t,
                    nclones=nclones,figfile='example-K14X40T-bf-and-clones-aei.png')

In [ ]:
#this TNO happens to be a plutino, so you can also plot the resonant evolution
#first we go back to the simulation archive to calcaulte the resonant angle
#where for plutinos the p:q resonances is 3:2
(flag, a, e, inc, node, aperi, ma, phi, t,res_str) = sbd.read_sa_for_resonance(
                            small_body=small_body,
                            archivefile=savefile,planet='neptune',
                            p=3,q=2,nclones=nclones)
flag, fig3 = sbd.plot_resonance(small_body=small_body,
                          res_string=res_str,
                          a=a,e=e,inc=inc,phi=phi,t=t,
                          nclones=nclones,figfile=None)